In [2]:
import re
import pandas as pd
from tqdm import tqdm


combined_regex = '^(?P<client>\S+) \S+ (?P<userid>\S+) \[(?P<datetime>[^\]]+)\] "(?P<method>[A-Z]+) (?P<request>[^ "]+)? HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+|-) "(?P<referrer>[^"]*)" "(?P<useragent>[^"]*)'
columns = [
    "client",
    "userid",
    "datetime",
    "method",
    "request",
    "status",
    "size",
    "referer",
    "user-agent",
]


In [3]:
def log_to_df(filename, output, error):
    with open(filename, "r") as file:
        line_num = 0
        parsed_lines = []
        for line in tqdm(file):
            try:
                cur_line = re.findall(combined_regex, line)[0]
                parsed_lines.append(cur_line)
            except Exception as e:
                with open(error, 'at') as err:
                    print((line, str(e)), file=err)
                continue

            line_num += 1

            if line_num % 250_000 == 0:
                df = pd.DataFrame(parsed_lines, columns=columns)
                df.to_parquet(f"{output}/file{line_num//250_000}.parquet")
                parsed_lines.clear()

        else:  # Handle the last batch
            df = pd.DataFrame(parsed_lines, columns=columns)
            df.to_parquet(f"{output}/file{line_num//250_000}.parquet")
            parsed_lines.clear()

In [4]:
log_to_df("access.log", "parquet/", "error.txt")


10365152it [00:49, 208731.60it/s]


In [5]:
df = pd.read_parquet("parquet/")

df['datetime'] = pd.to_datetime(df['datetime'], format='%d/%b/%Y:%H:%M:%S %z')
df['status'] = df['status'].astype(int)
df['size'] = pd.to_numeric(df['size'], errors='coerce')

df.dropna(inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10114865 entries, 0 to 10114864
Data columns (total 9 columns):
 #   Column      Dtype                    
---  ------      -----                    
 0   client      object                   
 1   userid      object                   
 2   datetime    datetime64[ns, UTC+03:30]
 3   method      object                   
 4   request     object                   
 5   status      int64                    
 6   size        int64                    
 7   referer     object                   
 8   user-agent  object                   
dtypes: datetime64[ns, UTC+03:30](1), int64(2), object(6)
memory usage: 694.5+ MB


In [6]:
df.head()

,client,userid,datetime,method,request,status,size,referer,user-agent
0,54.36.149.41,-,2019-01-22 03:56:14+03:30,GET,/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C...,200,30577,-,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...
1,31.56.96.51,-,2019-01-22 03:56:16+03:30,GET,/image/60844/productModel/200x200,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...
2,31.56.96.51,-,2019-01-22 03:56:16+03:30,GET,/image/61474/productModel/200x200,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...
3,40.77.167.129,-,2019-01-22 03:56:17+03:30,GET,/image/14925/productModel/100x100,200,1696,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...
4,91.99.72.15,-,2019-01-22 03:56:17+03:30,GET,/product/31893/62100/%D8%B3%D8%B4%D9%88%D8%A7%...,200,41483,-,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...


In [7]:
sqli_patterns = [
    "';--", "' OR '1'='1", "' OR 1=1 --", "' OR 0=0 --", "' OR x=x --",
    "admin' --", "') OR ('", "'; EXEC", "' UNION SELECT", "'; DROP", "' AND SLEEP("
]

sqli_patterns = [pattern.lower() for pattern in sqli_patterns]

df['sqli_flag'] = df['request'].str.lower().apply(
    lambda x: any(pattern in x for pattern in sqli_patterns)
)

sus_sqli = df[df['sqli_flag']]
sus_sqli

,client,userid,datetime,method,request,status,size,referer,user-agent,sqli_flag


In [8]:
xss_patterns = ['<script', 'javascript:', 'onerror=', 'onload=']
df['xss_flag'] = df['request'].apply(lambda x: any(pattern in x for pattern in xss_patterns))

sus_xss = df[df['xss_flag']]
sus_xss

,client,userid,datetime,method,request,status,size,referer,user-agent,sqli_flag,xss_flag
